# Exploring and Cleaning Player Team Data

In this notebook, we will be exploring and preparing a dataset containing player statistics from various teams. The overall goal is to clean and organize this data in a way that allows us to efficiently analyze and work with team-specific information.

### Step 1: Checking for Missing Values in Team Names

Our first step is to verify the integrity of the dataset by checking for any missing or undefined values in the `Basic Info_Team Name` column. Missing team names could indicate incomplete or faulty data, which could affect subsequent analysis. If any NaN values are found, we will either correct or remove them to ensure that the data is clean before proceeding with further analysis.


In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import MinMaxScaler
import pickle

# Read the data from the CSV file
df = pd.read_csv('../data/deep_player_data.csv')

In [2]:
# Filter the DataFrame (teams with missing or undefined team names)
filtered_df = df[df['Basic Info_Team Name'].isna() | (df['Basic Info_Team Name'] == 'undefined')]

# Select only the player name and team name columns
filtered_df_selected = filtered_df[['Basic Info_Player Name', 'Basic Info_Team Name']]

# Display the selected data
display(filtered_df_selected.head())
display(filtered_df_selected.tail())

,Basic Info_Player Name,Basic Info_Team Name
139,CLASIA,undefined
324,motm,undefined
562,chop,undefined
580,stamina,undefined


,Basic Info_Player Name,Basic Info_Team Name
139,CLASIA,undefined
324,motm,undefined
562,chop,undefined
580,stamina,undefined


In [3]:
# Get the count of rows in the filtered DataFrame
row_count = filtered_df_selected.shape[0]

# Display the row count
print(f'The number of players/teams in the filtered data: {row_count}')

The number of players/teams in the filtered data: 4


In [4]:
#Checking the original data frame
df_selected = df[['Basic Info_Player Name', 'Basic Info_Team Name']]
display(df_selected.head())

,Basic Info_Player Name,Basic Info_Team Name
0,ZywOo,Vitality
1,s1mple,Natus Vincere
2,sh1ro,Spirit
3,donk,Spirit
4,deko,Aurora


In [5]:
# Check the data types of the selected columns
team_name_dtype = df['Basic Info_Team Name'].dtype

# Check for NaN values in the 'Basic Info_Team Name' column and count occurrences of each type (NaN or not NaN)
team_name_nan_count = df['Basic Info_Team Name'].isna().sum()
team_name_non_nan_count = df['Basic Info_Team Name'].notna().sum()

# Display the results
print(f"Data type of 'Basic Info_Team Name': {team_name_dtype}")
print(f"Number of NaN values: {team_name_nan_count}")
print(f"Number of non-NaN values: {team_name_non_nan_count}")


Data type of 'Basic Info_Team Name': object
Number of NaN values: 0
Number of non-NaN values: 968


In [6]:
# Get unique values in the 'Basic Info_Team Name' column
unique_teams = df['Basic Info_Team Name'].unique()

# Count occurrences of each unique value
team_counts = df['Basic Info_Team Name'].value_counts(dropna=False)

# Display unique values and their counts
print("Unique team names and their counts:")
print(team_counts)

# Check for empty strings or whitespace-only entries
empty_entries = df[df['Basic Info_Team Name'].str.strip() == '']
print(f"\nNumber of empty or whitespace-only entries: {len(empty_entries)}")

# Check for entries that might indicate no team
no_team_entries = df[df['Basic Info_Team Name'].str.lower().isin(['no team', 'none', 'n/a', 'unknown'])]
print(f"\nNumber of entries potentially indicating no team: {len(no_team_entries)}")

# Check for unusually short team names (less than 2 characters)
short_names = df[df['Basic Info_Team Name'].str.len() < 2]
print(f"\nNumber of unusually short team names: {len(short_names)}")

# Display any unusual entries for manual inspection
print("\nUnusual entries (if any):")
unusual_entries = team_counts[team_counts < 5]
print(unusual_entries)

Unique team names and their counts:
Basic Info_Team Name
No team              383
Ninjas in Pyjamas      8
OG                     7
G2                     7
Into the Breach        7
                    ... 
Arcade                 1
MASONIC                1
ENCE Academy           1
Punishers              1
MIBR Academy           1
Name: count, Length: 185, dtype: int64

Number of empty or whitespace-only entries: 0

Number of entries potentially indicating no team: 383

Number of unusually short team names: 0

Unusual entries (if any):
Basic Info_Team Name
ESC             4
TSM             4
KRÜ             4
undefined       4
Liquid          4
               ..
Arcade          1
MASONIC         1
ENCE Academy    1
Punishers       1
MIBR Academy    1
Name: count, Length: 133, dtype: int64


### Step 2: Cleaning the Data

After confirming that no missing values (NaN) are present in the `Basic Info_Team Name` column only solo players listed under no team, we can move forward with cleaning and preparing the dataset. This involves handling outliers, normalizing numerical values, and ensuring that all columns are formatted consistently. 

We aim to normalize the player statistics across all teams to ensure consistent comparisons and analysis. Once the data is cleaned, we will split it into separate dataframes for each team, allowing us to analyze team-specific information more efficiently.


In [7]:
#Checking the original data frame data types
display(df.dtypes)

# Count the number of columns with each data type
type_counts = df.dtypes.value_counts()
print(type_counts)

Summary Stats_DPR                               float64
Summary Stats_KPR                               float64
Detailed Stats_Deaths / round                   float64
Role Stats_Firepower_Score                       object
Role Stats_Opening_Win% after opening kill       object
                                                 ...   
Detailed Stats_Maps played                        int64
Role Stats_Trading_Saved teammate per round     float64
Summary Stats_Rating 2.0                        float64
Role Stats_Opening_Score                         object
Role Stats_Entrying_Traded deaths percentage     object
Length: 74, dtype: object

float64    37
object     32
int64       5
Name: count, dtype: int64


In [8]:
type_dict = {str(t): df.select_dtypes(include=[t]).columns.tolist() for t in df.dtypes.unique()}
for t, cols in type_dict.items():
    print(f"\n{t}:")
    print(", ".join(cols))


float64:
Summary Stats_DPR, Summary Stats_KPR, Detailed Stats_Deaths / round, Detailed Stats_Damage / Round, Role Stats_Opening_Opening deaths per round, Detailed Stats_Assists / round, Role Stats_Sniping_Sniper kills per round, Role Stats_Opening_Opening kills per round, Role Stats_Sniping_Sniper multi-kill rounds, Detailed Stats_Kills / round, Detailed Stats_Rating 1.0, Detailed Stats_Rating 2.0, Role Stats_Clutching_Clutch points per round, Role Stats_Entrying_Saved by teammate per round, Role Stats_Utility_Utility kills per 100 rounds, Summary Stats_Impact, Role Stats_Utility_Flashes thrown per round, Role Stats_Firepower_Kills per round, Detailed Stats_Saved by teammate / round, Role Stats_Entrying_Traded deaths per round, Role Stats_Entrying_Assists per round, Role Stats_Trading_Trade kills per round, Role Stats_Utility_Flash assists per round, Role Stats_Utility_Time opponent flashed per round, Summary Stats_Rating 1.0, Summary Stats_ADR, Role Stats_Utility_Utility damage per r

In [9]:
# Check the number of unique values in each object column
for col in df.select_dtypes(include=['object']):
    print(f"\nColumn: {col}")
    print(df[col].head()) 
    print(df[col].nunique(), "unique values")


Column: Role Stats_Firepower_Score
0    98/100
1    97/100
2    85/100
3    98/100
4    89/100
Name: Role Stats_Firepower_Score, dtype: object
87 unique values

Column: Role Stats_Opening_Win% after opening kill
0    74.5%
1    75.6%
2    77.3%
3    73.8%
4    73.2%
Name: Role Stats_Opening_Win% after opening kill, dtype: object
111 unique values

Column: Role Stats_Clutching_Score
0    75/100
1    52/100
2    83/100
3    37/100
4    73/100
Name: Role Stats_Clutching_Score, dtype: object
59 unique values

Column: Detailed Stats_Headshot %
0    41.3%
1    41.1%
2    28.9%
3    60.6%
4    38.3%
Name: Detailed Stats_Headshot %, dtype: object
327 unique values

Column: Role Stats_Entrying_Support rounds
0    15.3%
1    10.5%
2    19.4%
3    12.4%
4    16.3%
Name: Role Stats_Entrying_Support rounds, dtype: object
134 unique values

Column: Basic Info_Team Name
0         Vitality
1    Natus Vincere
2           Spirit
3           Spirit
4           Aurora
Name: Basic Info_Team Name, dtype: o

In [10]:
def check_problematic_values(df, columns_to_check):
    results = {}
    for col in columns_to_check:
        if col in df.columns:
            total_count = len(df[col])
            nan_count = df[col].isna().sum()
            dash_count = (df[col] == '-').sum()
            other_count = total_count - nan_count - dash_count
            
            results[col] = {
                'NaN count': nan_count,
                'NaN percentage': (nan_count / total_count) * 100,
                '"-" count': dash_count,
                '"-" percentage': (dash_count / total_count) * 100,
                'Other values count': other_count,
                'Other values percentage': (other_count / total_count) * 100
            }
        else:
            results[col] = "Column not found in dataframe"
    
    return results

# List of columns to check
columns_to_check = [
    'Detailed Stats_Rating 1.0',
    'Summary Stats_Rating 1.0',
    'Detailed Stats_Rating 2.0',
    'Role Stats_Firepower_Rating 2.0',
    'Summary Stats_Rating 2.0',
    'Role Stats_Firepower_Pistol round rating',
    'Basic Info_Age'
]

# Run the check
results = check_problematic_values(df, columns_to_check)

# Display results
for col, data in results.items():
    print(f"\nColumn: {col}")
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, float):
                print(f"  {key}: {value:.2f}")
            else:
                print(f"  {key}: {value}")
    else:
        print(f"  {data}")



Column: Detailed Stats_Rating 1.0
  NaN count: 556
  NaN percentage: 57.44
  "-" count: 0
  "-" percentage: 0.00
  Other values count: 412
  Other values percentage: 42.56

Column: Summary Stats_Rating 1.0
  NaN count: 556
  NaN percentage: 57.44
  "-" count: 0
  "-" percentage: 0.00
  Other values count: 412
  Other values percentage: 42.56

Column: Detailed Stats_Rating 2.0
  NaN count: 412
  NaN percentage: 42.56
  "-" count: 0
  "-" percentage: 0.00
  Other values count: 556
  Other values percentage: 57.44

Column: Role Stats_Firepower_Rating 2.0
  NaN count: 0
  NaN percentage: 0.00
  "-" count: 412
  "-" percentage: 42.56
  Other values count: 556
  Other values percentage: 57.44

Column: Summary Stats_Rating 2.0
  NaN count: 412
  NaN percentage: 42.56
  "-" count: 0
  "-" percentage: 0.00
  Other values count: 556
  Other values percentage: 57.44

Column: Role Stats_Firepower_Pistol round rating
  NaN count: 0
  NaN percentage: 0.00
  "-" count: 412
  "-" percentage: 42.56
  

In [11]:
# Display a few rows of these columns for visual inspection
print("\nSample data:")
print(df[columns_to_check].head())


Sample data:
   Detailed Stats_Rating 1.0  Summary Stats_Rating 1.0  \
0                        NaN                       NaN   
1                       1.24                      1.24   
2                        NaN                       NaN   
3                        NaN                       NaN   
4                        NaN                       NaN   

   Detailed Stats_Rating 2.0 Role Stats_Firepower_Rating 2.0  \
0                       1.32                            1.29   
1                        NaN                               -   
2                       1.25                            1.22   
3                       1.27                            1.28   
4                       1.22                            1.17   

   Summary Stats_Rating 2.0 Role Stats_Firepower_Pistol round rating  \
0                      1.32                                     1.42   
1                       NaN                                        -   
2                      1.25         

In [12]:
# Drop the URL column
df = df.drop('URL', axis=1)
print("URL column has been dropped.")

URL column has been dropped.


In [13]:
# Handle strange age data
anomalous_age_row = df[df['Basic Info_Age'].astype(str).str.contains('years')]

if not anomalous_age_row.empty:
    # Extract the correct age from the anomalous entry
    anomalous_age = anomalous_age_row['Basic Info_Age'].iloc[0]
    correct_age = int(re.search(r'(\d+) years', str(anomalous_age)).group(1))
    
    # Update the age for this specific row
    df.loc[anomalous_age_row.index, 'Basic Info_Age'] = correct_age
    
    print(f"Updated age for player: {anomalous_age_row['Basic Info_Player Name'].iloc[0]}")
    print(f"Old value: {anomalous_age}")
    print(f"New value: {correct_age}")
else:
    print("No anomalous age entry found.")

# Verify the change
print("\nAge column summary after update:")
print(df['Basic Info_Age'].describe())

print("\nCheck for any remaining non-numeric values in the Age column:")
non_numeric_ages = df[pd.to_numeric(df['Basic Info_Age'], errors='coerce').isna()]
if non_numeric_ages.empty:
    print("No non-numeric values found in the Age column.")
else:
    print("Rows with non-numeric ages:")
    print(non_numeric_ages[['Basic Info_Player Name', 'Basic Info_Age']])

Updated age for player: ZywOo
Old value: 23 years
New value: 23

Age column summary after update:
count     968
unique      1
top        23
freq      968
Name: Basic Info_Age, dtype: int64

Check for any remaining non-numeric values in the Age column:
No non-numeric values found in the Age column.


In [14]:
def is_numeric_column(series):
    numeric_sample = series.dropna().sample(min(len(series), 100))
    return all(isinstance(x, str) and (
        ('/' in x) or 
        (x.replace('.', '').replace('-', '').isdigit()) or 
        (x.rstrip('%').replace('.', '').replace('-', '').isdigit()) or
        (x.split()[0].replace('.', '').replace('-', '').isdigit()) or  # For handling "23 years" format
        x.strip() == '-'  # Handle lone dash
    ) for x in numeric_sample)

def convert_to_numeric(value):
    if isinstance(value, str):
        value = value.strip()
        if '/' in value:
            numerator, denominator = value.split('/')
            return float(numerator) / float(denominator)
        elif value.endswith('%'):
            return float(value.rstrip('%')) / 100
        elif value.endswith('years'):
            return float(value.split()[0])
        elif value == '-':
            return np.nan
        else:
            try:
                return float(value)
            except ValueError:
                return np.nan
    return value

def process_dataframe(df):
    object_columns = df.select_dtypes(include=['object']).columns
    
    for col in object_columns:
        if is_numeric_column(df[col]) or 'Rating' in col or col == 'Basic Info_Age':
            print(f"Converting column: {col}")
            df[col] = df[col].apply(convert_to_numeric)
        else:
            print(f"Skipping non-numeric column: {col}")
    
    return df

def display_column_info(df):
    for col in df.columns:
        print(f"\nColumn: {col}")
        print(f"Data type: {df[col].dtype}")
        print(df[col].head())
        if df[col].dtype != 'object':
            print(f"Min value: {df[col].min()}, Max value: {df[col].max()}")
        print(f"Unique values: {df[col].nunique()}")

In [15]:
# Usage
df = process_dataframe(df)
display_column_info(df)

Converting column: Role Stats_Firepower_Score
Converting column: Role Stats_Opening_Win% after opening kill
Converting column: Role Stats_Clutching_Score
Converting column: Detailed Stats_Headshot %
Converting column: Role Stats_Entrying_Support rounds
Skipping non-numeric column: Basic Info_Team Name
Converting column: Role Stats_Sniping_Rounds with sniper kills percentage
Converting column: Role Stats_Firepower_Rating 2.0
Converting column: Role Stats_Opening_Opening success
Converting column: Role Stats_Firepower_Rounds with a kill
Converting column: Role Stats_Firepower_Rounds with a multi-kill
Converting column: Role Stats_Entrying_Score
Converting column: Basic Info_Age
Converting column: Role Stats_Trading_Trade kills percentage
Converting column: Role Stats_Clutching_Saves per round loss
Converting column: Role Stats_Trading_Score
Converting column: Role Stats_Entrying_Opening deaths traded percentage
Converting column: Summary Stats_KAST
Converting column: Role Stats_Trading_A

In [16]:
# List of columns that were missed in the previous conversion
missed_columns = [
    'Role Stats_Trading_Assisted kills percentage',#0    16.1%
    'Role Stats_Firepower_Pistol round rating', #0    1.42
    'Role Stats_Clutching_Time alive per round', #0    1m 10s
]

def convert_percentage(value):
    if isinstance(value, str) and value.endswith('%'):
        return float(value.rstrip('%')) / 100
    return value

def convert_rating(value):
    if isinstance(value, str):
        if value == '-':
            return np.nan
        else:
            return float(value)
    return value

def convert_time(value):
    if isinstance(value, str):
        parts = value.split()
        minutes = int(parts[0].rstrip('m'))
        seconds = int(parts[1].rstrip('s'))
        return minutes * 60 + seconds  # Convert to total seconds
    return value

# Convert the missed columns
for col in missed_columns:
    if col in df.columns:
        print(f"Converting column: {col}")
        if 'percentage' in col.lower():
            df[col] = df[col].apply(convert_percentage)
        elif 'rating' in col.lower():
            df[col] = df[col].apply(convert_rating)
        elif 'time' in col.lower():
            df[col] = df[col].apply(convert_time)
        
        print(f"New data type: {df[col].dtype}")
        print(df[col].head())
        print(f"Min value: {df[col].min()}, Max value: {df[col].max()}")
        print(f"Unique values: {df[col].nunique()}\n")
    else:
        print(f"Column not found: {col}\n")

Converting column: Role Stats_Trading_Assisted kills percentage
New data type: float64
0    0.161
1    0.081
2    0.100
3    0.181
4    0.077
Name: Role Stats_Trading_Assisted kills percentage, dtype: float64
Min value: -0.128, Max value: 0.259
Unique values: 228

Converting column: Role Stats_Firepower_Pistol round rating
New data type: float64
0    1.42
1     NaN
2    1.33
3    1.42
4    1.23
Name: Role Stats_Firepower_Pistol round rating, dtype: float64
Min value: 0.8, Max value: 1.42
Unique values: 59

Converting column: Role Stats_Clutching_Time alive per round
New data type: int64
0    70
1    59
2    79
3    66
4    80
Name: Role Stats_Clutching_Time alive per round, dtype: int64
Min value: -193652, Max value: 2380
Unique values: 89



In [17]:
# Check for remaining object columns
object_columns = df.select_dtypes(include=['object']).columns
if not object_columns.empty:
    print("\nRemaining object columns:")
    print(object_columns.tolist())
else:
    print("\nNo remaining object columns.")


Remaining object columns:
['Basic Info_Team Name', 'Basic Info_Real Name', 'Basic Info_Player Name']


### Step 3: Handling NaN Values and Dropping Unnecessary Columns

After converting all relevant columns to numeric data types (either `float` or `int`), we are left with only the necessary non-numeric columns, such as player names and team names, which remain as `object` types.

*Note: The `URL` column was dropped earlier to simplify the conversion process, as it was not needed for the analysis.*

In this step, we will:
1. **Check for NaN or missing values**: It’s important to ensure there are no missing values in the key columns, such as player statistics or team names, which could negatively impact our analysis.
2. **Evaluate and drop unnecessary columns**: We will review the remaining columns to identify any that are not useful for analysis, such as redundant or irrelevant fields, and remove them from the dataset to streamline our work.

By cleaning up the dataset, we will ensure it is ready for further analysis, and we can proceed with normalizing the data and performing team-based breakdowns.

In [18]:
# Function to calculate the percentage of missing values
def missing_percentage(column):
    return (column.isnull().sum() / len(column)) * 100

# Create a summary of missing values
missing_summary = df.apply(missing_percentage).sort_values(ascending=False)

# Filter columns with missing values
columns_with_missing = missing_summary[missing_summary > 0]

if not columns_with_missing.empty:
    print("Columns with missing values:")
    for column, percentage in columns_with_missing.items():
        print(f"{column}: {percentage:.2f}% missing")
else:
    print("No columns with missing values found.")

Columns with missing values:
Detailed Stats_Rating 1.0: 57.44% missing
Summary Stats_Rating 1.0: 57.44% missing
Detailed Stats_Rating 2.0: 42.56% missing
Role Stats_Firepower_Rating 2.0: 42.56% missing
Summary Stats_Rating 2.0: 42.56% missing
Role Stats_Firepower_Pistol round rating: 42.56% missing


In [19]:
# Check for any remaining non-numeric values that might have been converted to NaN
def check_nan_values(column):
    return column.isnull().sum() if column.dtype in ['float64', 'int64'] else 0

non_numeric_summary = df.apply(check_nan_values)
non_numeric_columns = non_numeric_summary[non_numeric_summary > 0]

if not non_numeric_columns.empty:
    print("\nNumeric columns with NaN values:")
    for column, count in non_numeric_columns.items():
        print(f"{column}: {count} NaN values")
else:
    print("\nNo numeric columns with NaN values found.")


Numeric columns with NaN values:
Detailed Stats_Rating 1.0: 556 NaN values
Role Stats_Firepower_Rating 2.0: 412 NaN values
Detailed Stats_Rating 2.0: 412 NaN values
Summary Stats_Rating 1.0: 556 NaN values
Role Stats_Firepower_Pistol round rating: 412 NaN values
Summary Stats_Rating 2.0: 412 NaN values


In [20]:
# Suggestion for handling missing values
print("\nSuggestions for handling missing values:")
for column, percentage in columns_with_missing.items():
    if percentage < 5:
        print(f"- {column}: Consider imputing missing values (e.g., mean, median, or mode).")
    elif 5 <= percentage < 20:
        print(f"- {column}: Investigate the reason for missing data. Consider advanced imputation techniques or feature engineering.")
    else:
        print(f"- {column}: High percentage of missing data. Consider dropping this column or using it to create a binary 'is_missing' feature.")


Suggestions for handling missing values:
- Detailed Stats_Rating 1.0: High percentage of missing data. Consider dropping this column or using it to create a binary 'is_missing' feature.
- Summary Stats_Rating 1.0: High percentage of missing data. Consider dropping this column or using it to create a binary 'is_missing' feature.
- Detailed Stats_Rating 2.0: High percentage of missing data. Consider dropping this column or using it to create a binary 'is_missing' feature.
- Role Stats_Firepower_Rating 2.0: High percentage of missing data. Consider dropping this column or using it to create a binary 'is_missing' feature.
- Summary Stats_Rating 2.0: High percentage of missing data. Consider dropping this column or using it to create a binary 'is_missing' feature.
- Role Stats_Firepower_Pistol round rating: High percentage of missing data. Consider dropping this column or using it to create a binary 'is_missing' feature.


In [21]:
def find_player_with_nan_ratings(df):
    """
    Find and display a player with NaN rating in the 'Summary Stats_Rating 2.0' column.
    
    :param df: pandas DataFrame
    :return: None (prints information)
    """
    # Specify the column we're interested in
    target_column = 'Summary Stats_Rating 2.0'
    
    # Find rows where the target column is NaN
    players_with_nan = df[df[target_column].isna()]
    
    if players_with_nan.empty:
        print(f"No players found with NaN ratings in the '{target_column}' column.")
        return
    
    # Select the first player with NaN rating
    player = players_with_nan.iloc[0]
    
    print(f"Player found with NaN rating in '{target_column}':")
    print(f"Name: {player['Basic Info_Player Name']}")
    print(f"Team: {player['Basic Info_Team Name']}")
    print(f"\n{target_column}: NaN")
    
    print("\nFull Player Information:")
    for col, value in player.items():
        print(f"{col}: {value}")

find_player_with_nan_ratings(df)

Player found with NaN rating in 'Summary Stats_Rating 2.0':
Name: s1mple
Team: Natus Vincere

Summary Stats_Rating 2.0: NaN

Full Player Information:
Summary Stats_DPR: 0.63
Summary Stats_KPR: 0.85
Detailed Stats_Deaths / round: 0.63
Role Stats_Firepower_Score: 0.97
Role Stats_Opening_Win% after opening kill: 0.7559999999999999
Role Stats_Clutching_Score: 0.52
Detailed Stats_Damage / Round: 85.4
Detailed Stats_Headshot %: 0.41100000000000003
Role Stats_Opening_Opening deaths per round: 0.09
Role Stats_Entrying_Support rounds: 0.105
Basic Info_Team Name: Natus Vincere
Role Stats_Sniping_Rounds with sniper kills percentage: 0.155
Detailed Stats_Assists / round: 0.1
Role Stats_Sniping_Sniper kills per round: 0.18
Role Stats_Opening_Opening kills per round: 0.11
Role Stats_Sniping_Sniper multi-kill rounds: 0.05
Detailed Stats_Kills / round: 0.85
Detailed Stats_Rating 1.0: 1.24
Role Stats_Firepower_Rating 2.0: nan
Role Stats_Opening_Opening success: 0.5529999999999999
Detailed Stats_Total k

In [22]:
# Removing duplicate colums
columns_to_drop = [
    'Role Stats_Firepower_Rating 2.0',
    'Detailed Stats_Rating 1.0',
    'Detailed Stats_Rating 2.0',
    'Role Stats_Firepower_Pistol round rating'
]

df = df.drop(columns=columns_to_drop)
print(f"Dropped columns: {columns_to_drop}")

Dropped columns: ['Role Stats_Firepower_Rating 2.0', 'Detailed Stats_Rating 1.0', 'Detailed Stats_Rating 2.0', 'Role Stats_Firepower_Pistol round rating']


In [23]:
# Combine the two rating columns (1.0 and 2.0) into a single column
df['Summary Stats_Rating'] = df['Summary Stats_Rating 1.0'].fillna(df['Summary Stats_Rating 2.0'])

# Create a binary indicator for missing ratings
df['Summary Stats_Rating_is_missing'] = df['Summary Stats_Rating'].isna().astype(int)

# Drop the original separate rating columns
df = df.drop(columns=['Summary Stats_Rating 1.0', 'Summary Stats_Rating 2.0'])

# Verify the changes
print("\nColumn names after changes:")
print(df.columns.tolist())

print("\nSummary of the combined rating column:")
print(df[['Summary Stats_Rating', 'Summary Stats_Rating_is_missing']].describe())

print("\nCheck for any remaining NaN values in the rating column:")
print(df['Summary Stats_Rating'].isna().sum())

print("\nSample of the updated data:")
print(df[['Summary Stats_Rating', 'Summary Stats_Rating_is_missing']].head())



Column names after changes:
['Summary Stats_DPR', 'Summary Stats_KPR', 'Detailed Stats_Deaths / round', 'Role Stats_Firepower_Score', 'Role Stats_Opening_Win% after opening kill', 'Role Stats_Clutching_Score', 'Detailed Stats_Damage / Round', 'Detailed Stats_Headshot %', 'Role Stats_Opening_Opening deaths per round', 'Role Stats_Entrying_Support rounds', 'Basic Info_Team Name', 'Role Stats_Sniping_Rounds with sniper kills percentage', 'Detailed Stats_Assists / round', 'Role Stats_Sniping_Sniper kills per round', 'Role Stats_Opening_Opening kills per round', 'Role Stats_Sniping_Sniper multi-kill rounds', 'Detailed Stats_Kills / round', 'Role Stats_Opening_Opening success', 'Detailed Stats_Total kills', 'Role Stats_Firepower_Rounds with a kill', 'Role Stats_Firepower_Rounds with a multi-kill', 'Detailed Stats_Total deaths', 'Role Stats_Entrying_Score', 'Role Stats_Clutching_Clutch points per round', 'Basic Info_Age', 'Role Stats_Trading_Trade kills percentage', 'Role Stats_Clutching_Sav

In [24]:
#Check for NaN values across all columns
nan_check = df.isna().sum()

# Filter to show only columns with NaN values
columns_with_nan = nan_check[nan_check > 0]

if columns_with_nan.empty:
    print("Great news! There are no NaN values in any column of the DataFrame.")
else:
    print("The following columns contain NaN values:")
    print(columns_with_nan)

Great news! There are no NaN values in any column of the DataFrame.


### Step 4: Final Data Preparation and Column Adjustments

With all NaN values successfully addressed and only the relevant object values (such as player names and team names) remaining in our dataset, we have now moved on to finalizing our data structure before performing any normalization.

In this step, we have:
1. **Renamed Columns**: To make our dataset more intuitive and easier to work with, we renamed the columns for readability and consistency.
2. **Reordered Columns**: The columns were reordered logically to group related statistics and ensure a clean structure for further analysis.
3. **Set Player Name as the Index**: We set the player names as the index to ensure that each player is easily identifiable and that our data is structured efficiently for team-based analysis.

With these changes complete, we are now ready to move on to the next phase, where we will normalize the numerical data and split it into team-based DataFrames.


In [25]:
# Print all column names
print("Current column names:")
for col in df.columns:
    print(col)

Current column names:
Summary Stats_DPR
Summary Stats_KPR
Detailed Stats_Deaths / round
Role Stats_Firepower_Score
Role Stats_Opening_Win% after opening kill
Role Stats_Clutching_Score
Detailed Stats_Damage / Round
Detailed Stats_Headshot %
Role Stats_Opening_Opening deaths per round
Role Stats_Entrying_Support rounds
Basic Info_Team Name
Role Stats_Sniping_Rounds with sniper kills percentage
Detailed Stats_Assists / round
Role Stats_Sniping_Sniper kills per round
Role Stats_Opening_Opening kills per round
Role Stats_Sniping_Sniper multi-kill rounds
Detailed Stats_Kills / round
Role Stats_Opening_Opening success
Detailed Stats_Total kills
Role Stats_Firepower_Rounds with a kill
Role Stats_Firepower_Rounds with a multi-kill
Detailed Stats_Total deaths
Role Stats_Entrying_Score
Role Stats_Clutching_Clutch points per round
Basic Info_Age
Role Stats_Trading_Trade kills percentage
Role Stats_Clutching_Saves per round loss
Role Stats_Entrying_Saved by teammate per round
Role Stats_Trading_Sc

In [26]:
#Renaming columns to simplify the names
column_mapping = {
    # Basic Info
    'Basic Info_Player Name': 'player_name',
    'Basic Info_Real Name': 'real_name',
    'Basic Info_Team Name': 'team',
    'Basic Info_Age': 'age',

    # Summary Stats
    'Summary Stats_Rating': 'rating',
    'Summary Stats_DPR': 'dpr',
    'Summary Stats_KPR': 'kpr',
    'Summary Stats_KAST': 'kast',
    'Summary Stats_Impact': 'impact',
    'Summary Stats_ADR': 'adr',
    'Summary Stats_Rating_is_missing': 'rating_is_missing',

    # Detailed Stats
    'Detailed Stats_Total kills': 'total_kills',
    'Detailed Stats_Total deaths': 'total_deaths',
    'Detailed Stats_Rounds played': 'rounds_played',
    'Detailed Stats_K/D Ratio': 'kd_ratio',
    'Detailed Stats_Maps played': 'maps_played',
    'Detailed Stats_Kills / round': 'kills_per_round',
    'Detailed Stats_Deaths / round': 'deaths_per_round',
    'Detailed Stats_Assists / round': 'assists_per_round',
    'Detailed Stats_Saved by teammate / round': 'saved_by_teammate_per_round',
    'Detailed Stats_Saved teammates / round': 'saved_teammates_per_round',
    'Detailed Stats_Damage / Round': 'damage_per_round',
    'Detailed Stats_Grenade dmg / Round': 'grenade_damage_per_round',
    'Detailed Stats_Headshot %': 'headshot_percentage',

    # Role Stats - Firepower
    'Role Stats_Firepower_Score': 'firepower_score',
    'Role Stats_Firepower_Kills per round': 'firepower_kills_per_round',
    'Role Stats_Firepower_Damage per round': 'firepower_damage_per_round',
    'Role Stats_Firepower_Kills per round win': 'firepower_kills_per_round_win',
    'Role Stats_Firepower_Damage per round win': 'firepower_damage_per_round_win',
    'Role Stats_Firepower_Rounds with a kill': 'firepower_rounds_with_kill',
    'Role Stats_Firepower_Rounds with a multi-kill': 'firepower_rounds_with_multi_kill',

    # Role Stats - Opening
    'Role Stats_Opening_Score': 'opening_score',
    'Role Stats_Opening_Opening kills per round': 'opening_kills_per_round',
    'Role Stats_Opening_Opening deaths per round': 'opening_deaths_per_round',
    'Role Stats_Opening_Opening attempts': 'opening_attempts',
    'Role Stats_Opening_Opening success': 'opening_success',
    'Role Stats_Opening_Win% after opening kill': 'win_percentage_after_opening_kill',
    'Role Stats_Opening_Attacks per round': 'opening_attacks_per_round',

    # Role Stats - Clutching
    'Role Stats_Clutching_Score': 'clutching_score',
    'Role Stats_Clutching_Clutch points per round': 'clutch_points_per_round',
    'Role Stats_Clutching_1on1 win percentage': 'clutch_1on1_win_percentage',
    'Role Stats_Clutching_Time alive per round': 'clutch_time_alive_per_round',
    'Role Stats_Clutching_Saves per round loss': 'clutch_saves_per_round_loss',
    'Role Stats_Clutching_Last alive percentage': 'clutch_last_alive_percentage',

    # Role Stats - Entrying
    'Role Stats_Entrying_Score': 'entrying_score',
    'Role Stats_Entrying_Saved by teammate per round': 'entrying_saved_by_teammate_per_round',
    'Role Stats_Entrying_Traded deaths per round': 'entrying_traded_deaths_per_round',
    'Role Stats_Entrying_Traded deaths percentage': 'entrying_traded_deaths_percentage',
    'Role Stats_Entrying_Opening deaths traded percentage': 'entrying_opening_deaths_traded_percentage',
    'Role Stats_Entrying_Assists per round': 'entrying_assists_per_round',
    'Role Stats_Entrying_Support rounds': 'entrying_support_rounds',

    # Role Stats - Trading
    'Role Stats_Trading_Score': 'trading_score',
    'Role Stats_Trading_Trade kills per round': 'trading_kills_per_round',
    'Role Stats_Trading_Trade kills percentage': 'trading_kills_percentage',
    'Role Stats_Trading_Assisted kills percentage': 'trading_assisted_kills_percentage',
    'Role Stats_Trading_Damage per kill': 'trading_damage_per_kill',
    'Role Stats_Trading_Saved teammate per round': 'trading_saved_teammate_per_round',

    # Role Stats - Sniping
    'Role Stats_Sniping_Score': 'sniping_score',
    'Role Stats_Sniping_Sniper kills per round': 'sniping_kills_per_round',
    'Role Stats_Sniping_Sniper kills percentage': 'sniping_kills_percentage',
    'Role Stats_Sniping_Rounds with sniper kills percentage': 'sniping_rounds_with_kills_percentage',
    'Role Stats_Sniping_Sniper multi-kill rounds': 'sniping_multi_kill_rounds',
    'Role Stats_Sniping_Sniper opening kills per round': 'sniping_opening_kills_per_round',

    # Role Stats - Utility
    'Role Stats_Utility_Score': 'utility_score',
    'Role Stats_Utility_Utility damage per round': 'utility_damage_per_round',
    'Role Stats_Utility_Utility kills per 100 rounds': 'utility_kills_per_100_rounds',
    'Role Stats_Utility_Flashes thrown per round': 'utility_flashes_thrown_per_round',
    'Role Stats_Utility_Flash assists per round': 'utility_flash_assists_per_round',
    'Role Stats_Utility_Time opponent flashed per round': 'utility_time_opponent_flashed_per_round'
}

# Apply the mapping to rename the columns
df.rename(columns=column_mapping, inplace=True)

In [27]:
# Reorder the columns based on the new grouping
column_order = [
    # Basic Info
    'player_name',
    'real_name',
    'team',
    'age',

    # Summary Stats
    'rating',
    'dpr',
    'kpr',
    'kast',
    'impact',
    'adr',
    'rating_is_missing',

    # Detailed Stats
    'total_kills',
    'total_deaths',
    'rounds_played',
    'kd_ratio',
    'maps_played',
    'kills_per_round',
    'deaths_per_round',
    'assists_per_round',
    'saved_by_teammate_per_round',
    'saved_teammates_per_round',
    'damage_per_round',
    'grenade_damage_per_round',
    'headshot_percentage',

    # Role Stats - Firepower
    'firepower_score',
    'firepower_kills_per_round',
    'firepower_damage_per_round',
    'firepower_kills_per_round_win',
    'firepower_damage_per_round_win',
    'firepower_rounds_with_kill',
    'firepower_rounds_with_multi_kill',

    # Role Stats - Opening
    'opening_score',
    'opening_kills_per_round',
    'opening_deaths_per_round',
    'opening_attempts',
    'opening_success',
    'win_percentage_after_opening_kill',
    'opening_attacks_per_round',

    # Role Stats - Clutching
    'clutching_score',
    'clutch_points_per_round',
    'clutch_1on1_win_percentage',
    'clutch_time_alive_per_round',
    'clutch_saves_per_round_loss',
    'clutch_last_alive_percentage',

    # Role Stats - Entrying
    'entrying_score',
    'entrying_saved_by_teammate_per_round',
    'entrying_traded_deaths_per_round',
    'entrying_traded_deaths_percentage',
    'entrying_opening_deaths_traded_percentage',
    'entrying_assists_per_round',
    'entrying_support_rounds',

    # Role Stats - Trading
    'trading_score',
    'trading_kills_per_round',
    'trading_kills_percentage',
    'trading_assisted_kills_percentage',
    'trading_damage_per_kill',
    'trading_saved_teammate_per_round',

    # Role Stats - Sniping
    'sniping_score',
    'sniping_kills_per_round',
    'sniping_kills_percentage',
    'sniping_rounds_with_kills_percentage',
    'sniping_multi_kill_rounds',
    'sniping_opening_kills_per_round',

    # Role Stats - Utility
    'utility_score',
    'utility_damage_per_round',
    'utility_kills_per_100_rounds',
    'utility_flashes_thrown_per_round',
    'utility_flash_assists_per_round',
    'utility_time_opponent_flashed_per_round'
]

# Apply the mapping to rename the columns
df.rename(columns=column_mapping, inplace=True)

# Reorder the DataFrame columns
df = df[column_order]

# Display the first few rows of the reorganized DataFrame
print(df.head())

# Print the shape of the DataFrame to confirm all columns are included
print(f"\nDataFrame shape: {df.shape}")

  player_name            real_name           team  age  rating   dpr   kpr  \
0       ZywOo      Mathieu Herbaut       Vitality   23    1.32  0.61  0.84   
1      s1mple  Oleksandr Kostyliev  Natus Vincere   23    1.24  0.63  0.85   
2       sh1ro       Dmitry Sokolov         Spirit   23    1.25  0.54  0.77   
3        donk    Danil Kryshkovets         Spirit   23    1.27  0.67  0.84   
4        deko         Denis Zhukov         Aurora   23    1.22  0.59  0.79   

    kast  impact   adr  ...  sniping_kills_percentage  \
0  0.754    1.43  87.3  ...                     0.269   
1  0.739    1.34  85.4  ...                     0.288   
2  0.763    1.20  78.3  ...                     0.467   
3  0.741    1.38  88.3  ...                     0.015   
4  0.747    1.18  79.8  ...                     0.315   

   sniping_rounds_with_kills_percentage  sniping_multi_kill_rounds  \
0                                 0.147                      0.040   
1                                 0.155         

In [28]:
# Check for duplicate player names
duplicate_players = df[df.duplicated(subset=['player_name'], keep=False)]

if duplicate_players.empty:
    print("No duplicate player names found. player_name can be safely used as the index.")
    df.set_index('player_name', inplace=True)
    print("player_name has been set as the index.")
else:
    print("Warning: Duplicate player names found. Here are the duplicates:")
    
    # Group by player name and display the duplicates
    for name, group in duplicate_players.groupby('player_name'):
        print(f"\nDuplicate entries for player: {name}")
        print(group[['real_name', 'team', 'age']].to_string(index=False))
    
    print("\nConsider using a different column or creating a unique identifier.")

# Display the first few rows of the DataFrame
print("\nFirst few rows of the DataFrame:")
print(df.head())


Duplicate entries for player: Lucky
     real_name  team  age
  Philip Ewald Sashi   23
Lucas Chastang 3DMAX   23

Duplicate entries for player: steel
    real_name    team  age
Joshua Nissan    Vibe   23
  Lucas Lopes No team   23

Consider using a different column or creating a unique identifier.

First few rows of the DataFrame:
  player_name            real_name           team  age  rating   dpr   kpr  \
0       ZywOo      Mathieu Herbaut       Vitality   23    1.32  0.61  0.84   
1      s1mple  Oleksandr Kostyliev  Natus Vincere   23    1.24  0.63  0.85   
2       sh1ro       Dmitry Sokolov         Spirit   23    1.25  0.54  0.77   
3        donk    Danil Kryshkovets         Spirit   23    1.27  0.67  0.84   
4        deko         Denis Zhukov         Aurora   23    1.22  0.59  0.79   

    kast  impact   adr  ...  sniping_kills_percentage  \
0  0.754    1.43  87.3  ...                     0.269   
1  0.739    1.34  85.4  ...                     0.288   
2  0.763    1.20  78.3  .

In [29]:
# Set the player_name column as the index - duplicate player names are allowed and differentiated by real_name
df.set_index('player_name', inplace=True, verify_integrity=False)

In [30]:
# Adjust the column_order list to remove 'player_name' since it's now the index
column_order.remove('player_name')

# Reorder the DataFrame columns
df = df[column_order]

# Display the first few rows of the reorganized DataFrame
print(df.head())

# Print the shape of the DataFrame to confirm all columns are included
print(f"\nDataFrame shape: {df.shape}")

                       real_name           team  age  rating   dpr   kpr  \
player_name                                                                
ZywOo            Mathieu Herbaut       Vitality   23    1.32  0.61  0.84   
s1mple       Oleksandr Kostyliev  Natus Vincere   23    1.24  0.63  0.85   
sh1ro             Dmitry Sokolov         Spirit   23    1.25  0.54  0.77   
donk           Danil Kryshkovets         Spirit   23    1.27  0.67  0.84   
deko                Denis Zhukov         Aurora   23    1.22  0.59  0.79   

              kast  impact   adr  rating_is_missing  ...  \
player_name                                          ...   
ZywOo        0.754    1.43  87.3                  0  ...   
s1mple       0.739    1.34  85.4                  0  ...   
sh1ro        0.763    1.20  78.3                  0  ...   
donk         0.741    1.38  88.3                  0  ...   
deko         0.747    1.18  79.8                  0  ...   

             sniping_kills_percentage  sniping

### Step 5: Normalizing Data and Creating Team-Specific DataFrames

Now that the columns are renamed, ordered, and properly indexed, we will proceed with normalizing the numerical data to ensure consistent comparisons across all teams.

In this step, we will:
1. **Normalize Player Statistics**: The numerical columns (such as player statistics) will be normalized to bring all values to a comparable scale, ensuring meaningful analysis.
2. **Create Team-Specific DataFrames**: After normalization, we will split the data into separate DataFrames for each team, allowing us to focus on team-specific analysis.
3. **Handle Players Without a Team**: Any players without a team will be placed into a separate "solo player" DataFrame for further investigation or analysis.

This step will fully prepare the dataset for both team-based and individual analysis, ensuring that it is clean, normalized, and ready for deeper insights.

In [31]:
# Normalize the numerical data
# Select numerical columns (excluding 'age' and 'rating_is_missing')
numerical_columns = df.select_dtypes(include=[np.number]).columns.tolist()
numerical_columns = [col for col in numerical_columns if col not in ['age', 'rating_is_missing']]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the numerical data
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

In [32]:
# Standardize team names to lowercase
df['team'] = df['team'].str.lower()

# Sort by rating in descending order
df_sorted = df.sort_values('rating', ascending=False)

# Remove duplicate entries based on 'real_name', keeping the highest-rated entry
df_unique_players = df_sorted.drop_duplicates(subset='real_name', keep='first')

# Create data frames for each team
team_dfs = {}
for team in df['team'].unique():
    if team != 'no team':
        team_dfs[team] = df[df['team'] == team].copy()

In [33]:
# Create a data frame for solo players
solo_players_df = df_unique_players[df_unique_players['team'] == 'no team'].copy()

In [34]:
# Review the data frames
def summarize_data(df, name):
    summary = f"Summary for {name}:\n"
    summary += f"Number of players: {df['real_name'].nunique()}\n"
    summary += f"Average rating: {df['rating'].mean():.3f}\n"
    summary += f"Top 3 players by rating:\n"
    top_players = df.sort_values('rating', ascending=False).head(3)
    for idx, player in top_players.iterrows():
        summary += f"  - {player['player_name']} ({player['real_name']}): {player['rating']:.3f}\n"
    summary += f"Key stats (mean values):\n"
    for stat in ['kpr', 'dpr', 'adr', 'kast']:
        summary += f"  - {stat.upper()}: {df[stat].mean():.3f}\n"
    return summary

# Summarize team data
print(f"Number of teams: {len(team_dfs)}")
print(f"Number of solo players: {len(solo_players_df)}")

# Select a few teams to summarize
teams_to_summarize = list(team_dfs.keys())[:5]  # First 5 teams
for team in teams_to_summarize:
    team_df = team_dfs[team].reset_index()  # Ensure 'player_name' is a column
    print("\n" + summarize_data(team_df, team))

# Summarize solo player data
solo_players_df_reset = solo_players_df.reset_index()  # Ensure 'player_name' is a column
print("\n" + summarize_data(solo_players_df_reset, "Solo Players"))

# Overall dataset statistics
all_players = pd.concat(list(team_dfs.values()) + [solo_players_df]).reset_index()  # Ensure 'player_name' is a column
print("\nOverall Dataset Statistics:")
print(f"Total number of players: {all_players['real_name'].nunique()}")
print(f"Average rating across all players: {all_players['rating'].mean():.3f}")
highest_rated_player = all_players.loc[all_players['rating'].idxmax()]
lowest_rated_player = all_players.loc[all_players['rating'].idxmin()]
print(f"Highest rated player: {highest_rated_player['player_name']} ({highest_rated_player['real_name']}) (Rating: {highest_rated_player['rating']:.3f})")
print(f"Lowest rated player: {lowest_rated_player['player_name']} ({lowest_rated_player['real_name']}) (Rating: {lowest_rated_player['rating']:.3f})")

Number of teams: 184
Number of solo players: 383

Summary for vitality:
Number of players: 5
Average rating: 0.639
Top 3 players by rating:
  - ZywOo (Mathieu Herbaut): 1.000
  - Spinx (Lotan Giladi): 0.627
  - flameZ (Shahar Shushan): 0.608
Key stats (mean values):
  - KPR: 0.607
  - DPR: 0.505
  - ADR: 0.603
  - KAST: 0.676


Summary for natus vincere:
Number of players: 7
Average rating: 0.524
Top 3 players by rating:
  - s1mple (Oleksandr Kostyliev): 0.843
  - w0nderful (Ihor Zhdanov): 0.765
  - jL (Justinas Lekavicius): 0.608
Key stats (mean values):
  - KPR: 0.538
  - DPR: 0.544
  - ADR: 0.506
  - KAST: 0.569


Summary for spirit:
Number of players: 5
Average rating: 0.620
Top 3 players by rating:
  - donk (Danil Kryshkovets): 0.902
  - sh1ro (Dmitry Sokolov): 0.863
  - zont1x (Myroslav Plakhotia): 0.569
Key stats (mean values):
  - KPR: 0.547
  - DPR: 0.419
  - ADR: 0.519
  - KAST: 0.733


Summary for aurora:
Number of players: 5
Average rating: 0.553
Top 3 players by rating:
  

### Step 6: Saving the Cleaned Data
We have now normalized our data and created separate data frames for each team and for solo players.
After addressing challenges such as duplicate entries, inconsistent team names, and managing players without a team, we’re now ready to save the finalized datasets.

We’ll pickle the data for easy and consistent access across the team:
- **Team DataFrames**: Stored as a pickle file for team-based analysis.
- **Solo Players DataFrame**: Pickled separately for individual player insights.
- **Full Cleaned DataFrame**: Saved as a master pickle file for shared use across models.

This ensures everyone can load the same, clean data for further analysis and modeling.

In [35]:
# Save the team DataFrames dictionary
with open('team_dfs.pkl', 'wb') as f:
    pickle.dump(team_dfs, f)

In [36]:
# Save the solo players DataFrame
with open('solo_players_df.pkl', 'wb') as f:
    pickle.dump(solo_players_df, f)


In [37]:
# Save the full cleaned DataFrame
with open('all_players_df.pkl', 'wb') as f:
    pickle.dump(all_players, f)
